In [4]:
!pip install --upgrade setuptools pip

     |████████████████████████████████| 798kB 2.7MB/s 
     |████████████████████████████████| 1.5MB 13.6MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: setuptools 49.1.0
    Uninstalling setuptools-49.1.0:
      Successfully uninstalled setuptools-49.1.0
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1


In [3]:
!pip install rake-nltk

In [15]:
import requests
import nltk
from rake_nltk import Rake
nltk.download('stopwords')
from nltk.sentiment.vader import SentimentIntensityAnalyzer 
from bs4 import BeautifulSoup
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import random
from wordcloud import WordCloud
from html.parser import HTMLParser
import heapq
import bs4 as bs
import urllib.request
import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Extracting the Article 
#### Article being worked on : [InvestorPlace : Buy Covanta Stock Like Its CEO](https://investorplace.com/2015/03/buy-covanta-stock-like-ceo/)

In [7]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
url = 'https://investorplace.com/2015/03/buy-covanta-stock-like-ceo/'
webpage = requests.get(url,headers=headers).text
webpage

'<!DOCTYPE html>\n<html lang="en-US">\n\n<head>\n\t<title>\n\t\tCVA: Buy Covanta Stock Like Its CEO | InvestorPlace\t</title>\n\n\t<link rel="profile" href="http://gmpg.org/xfn/11" />\n\t\n\t<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">\n\t<meta http-equiv="X-UA-Compatible" content="IE=edge"><script type="text/javascript">(window.NREUM||(NREUM={})).loader_config={licenseKey:"37729af97b",applicationID:"117864961"};window.NREUM||(NREUM={}),__nr_require=function(e,n,t){function r(t){if(!n[t]){var i=n[t]={exports:{}};e[t][0].call(i.exports,function(n){var i=e[t][1][n];return r(i||n)},i,i.exports)}return n[t].exports}if("function"==typeof __nr_require)return __nr_require;for(var i=0;i<t.length;i++)r(t[i]);return r}({1:[function(e,n,t){function r(){}function i(e,n,t){return function(){return o(e,[u.now()].concat(f(arguments)),n?null:this,t),n?void 0:this}}var o=e("handle"),a=e(4),f=e(5),c=e("ee").get("tracer"),u=e("loader"),s=NREUM;"undefined"==typeof window.newrelic&&(

In [8]:
# Scraping the text
article_text = []
soup = BeautifulSoup(webpage,'lxml')
paragraphs = []
article = soup.find_all('p')
# article[8:]
# 8,10,11(extract <span></span>),12,13,15,16,17,19,21 (noted down the useful lines)
for i in [8,10,11,12,13,15,16,17,19,21]:
  if i == 11:
    article_text.append(article[i].find('span').text.strip())
  else:
    article_text.append(article[i].text.strip())
print("Extracted short article : \n{}".format(" ".join(article_text)))

Extracted short article : 
A company’s own top management tend to have the best inside view into the business. So, when company officers make major buys, investors are wise to take notice. Maybe they find the stock very undervalued, or maybe they see exciting progress within the company, or maybe both. So, in this series we look at the largest insider buys by the ”top brass” over the trailing six-month period, one of which was a total of $1.1 million by Stephen J. Jones, CEO at Jones’s average cost works out to $21.15 per share. Shares of Covanta were changing hands at $21.54 at last check. The chart below shows the one-year performance of CVA stock versus its 200-day moving average: Looking at the chart above, CVA’s low point in its 52-week range is $16.42 per share, with $25.35 as the 52-week high point — that compares with a last trade of $21.54. The current annualized dividend paid by Covanta is $1 per share, currently paid in quarterly installments, and its most recent dividend ha

### Converting the entire article to lowercase and removing the stopwords

In [13]:
art = " ".join(article_text)
art = art.lower().split()
stopwords = nltk.corpus.stopwords.words('english')
final_words = [w for w in art if w not in stopwords]
final_art = " ".join(final_words)
final_art

'company’s top management tend best inside view business. so, company officers make major buys, investors wise take notice. maybe find stock undervalued, maybe see exciting progress within company, maybe both. so, series look largest insider buys ”top brass” trailing six-month period, one total $1.1 million stephen j. jones, ceo jones’s average cost works $21.15 per share. shares covanta changing hands $21.54 last check. chart shows one-year performance cva stock versus 200-day moving average: looking chart above, cva’s low point 52-week range $16.42 per share, $25.35 52-week high point — compares last trade $21.54. current annualized dividend paid covanta $1 per share, currently paid quarterly installments, recent dividend upcoming ex-date mar. 27. long-term dividend history chart cva, good help judging whether recent dividend approximately 4.7% annualized yield likely continue. according etf finder etf channel, cva makes 4.4% ishares global clean energy etf (nasdaq:icln). article pri

### Extracting keywords and their corresponding words scores from the above article using the inbuilt rake_nltk package (**RAKE** : **R**apid **A**utomatic **K**eyword **E**xtraction)

In [19]:
extracted_dict = {}
rake = Rake()
rake.extract_keywords_from_text(final_art)
scores_keyw = rake.get_ranked_phrases_with_scores()
scores_keyw
# Create a dictionary
keys = [elem[1] for elem in scores_keyw]
values = [elem[0] for elem in scores_keyw]
for i in range(len(keys)):
  extracted_dict[keys[i]] = values[i]
extracted_dict

{'03': 1.0,
 '1': 2.6666666666666665,
 '1 million stephen j': 14.666666666666666,
 '1 per share': 8.666666666666666,
 '15 per share': 9.0,
 '16': 1.0,
 '2015': 1.0,
 '21': 1.0,
 '25': 1.0,
 '27': 1.0,
 '35 52': 4.5,
 '4': 4.0,
 '42 per share': 9.0,
 '54': 2.0,
 '54 last check': 10.0,
 '7': 1.0,
 'according etf finder etf channel': 25.0,
 'annualized yield likely continue': 16.5,
 'article printed investorplace media': 15.5,
 'average cost works': 9.0,
 'buy': 1.0,
 'ceo /.': 4.5,
 'ceo jones ’': 6.833333333333334,
 'chart shows one': 8.833333333333334,
 'com': 1.0,
 'company officers make major buys': 27.333333333333332,
 'company ’': 6.666666666666666,
 'covanta': 3.3333333333333335,
 'current annualized dividend paid covanta': 22.833333333333332,
 'currently paid quarterly installments': 16.5,
 'cva makes 4': 11.0,
 'cva ’': 6.333333333333334,
 'date mar': 4.0,
 'day moving average': 9.0,
 'good help judging whether recent dividend approximately 4': 55.5,
 'https :// investorplace': 

### Top 10 key ideas

In [23]:
key_10 = heapq.nlargest(10,extracted_dict,key = extracted_dict.get)
top10 = " ".join(key_10)
print("TOP 10 KEY IDEAS SUMMARIZED : \n")
top10

TOP 10 KEY IDEAS SUMMARIZED : 



'series look largest insider buys ” top brass ” trailing six good help judging whether recent dividend approximately 4 top management tend best inside view business week high point — compares last trade maybe see exciting progress within company year performance cva stock versus 200 company officers make major buys ishares global clean energy etf according etf finder etf channel term dividend history chart cva'

### Top 15 key ideas

In [24]:
key_15 = heapq.nlargest(15,extracted_dict,key = extracted_dict.get)
top15 = " ".join(key_15)
print("TOP 15 KEY IDEAS SUMMARIZED : \n")
top15

TOP 15 KEY IDEAS SUMMARIZED : 



'series look largest insider buys ” top brass ” trailing six good help judging whether recent dividend approximately 4 top management tend best inside view business week high point — compares last trade maybe see exciting progress within company year performance cva stock versus 200 company officers make major buys ishares global clean energy etf according etf finder etf channel term dividend history chart cva current annualized dividend paid covanta recent dividend upcoming ex currently paid quarterly installments annualized yield likely continue investors wise take notice'

### Top 20 key ideas

In [25]:
key_20 = heapq.nlargest(20,extracted_dict,key = extracted_dict.get)
top20 = " ".join(key_20)
print("TOP 20 KEY IDEAS SUMMARIZED : \n")
top20

TOP 20 KEY IDEAS SUMMARIZED : 



'series look largest insider buys ” top brass ” trailing six good help judging whether recent dividend approximately 4 top management tend best inside view business week high point — compares last trade maybe see exciting progress within company year performance cva stock versus 200 company officers make major buys ishares global clean energy etf according etf finder etf channel term dividend history chart cva current annualized dividend paid covanta recent dividend upcoming ex currently paid quarterly installments annualized yield likely continue investors wise take notice article printed investorplace media shares covanta changing hands maybe find stock undervalued 1 million stephen j cva makes 4'

### Top 30 key ideas

In [26]:
key_30 = heapq.nlargest(30,extracted_dict,key = extracted_dict.get)
top30 = " ".join(key_30)
print("TOP 30 KEY IDEAS SUMMARIZED : \n")
top30

TOP 30 KEY IDEAS SUMMARIZED : 



'series look largest insider buys ” top brass ” trailing six good help judging whether recent dividend approximately 4 top management tend best inside view business week high point — compares last trade maybe see exciting progress within company year performance cva stock versus 200 company officers make major buys ishares global clean energy etf according etf finder etf channel term dividend history chart cva current annualized dividend paid covanta recent dividend upcoming ex currently paid quarterly installments annualized yield likely continue investors wise take notice article printed investorplace media shares covanta changing hands maybe find stock undervalued 1 million stephen j cva makes 4 low point 52 54 last check https :// investorplace day moving average average cost works 42 per share 15 per share chart shows one 1 per share ceo jones ’'